# Set up HCP Vault

Sign up for HCP Vault if you haven't already: https://portal.cloud.hashicorp.com/

In this guide, we will automate the creation of our enviroments. For this we will need HCP credentials for a Service Principal to use with Terraform.

## Required Credentials

### HCP Vault Credentials

After signing up to Hashicorp Cloud Platform.
* Login at https://portal.cloud.hashicorp.com/sign-in
* Go to **Access control (IAM)** -> **Service Principals**.
* Create a Service Principal with the Contributor Role.
  * Click "**Create service principal**" and provide the following info:
	* Name: eg `pphan`
	* Role: `Contributor`
  * Click "Create service principal".
* Create key for service principal.
  * Click the service principal you just created.
  * Click "**Create service principle key**".
  * Save your "**Client ID**" and "**Client secret**". NOTE: You will not be able to see secret again.
* Export these credentials as enviroment variables:
```shell
export HCP_CLIENT_ID=<YOUR_HCP_SERVICE_PRINCIPAL_ID>
export HCP_CLIENT_SECRET=<YOU_HCP_SECRET_KEY>
```

In [ ]:
export HCP_CLIENT_ID=ZORhkMMPRN8ls6igRazYs5hUX8mlIRWS
export HCP_CLIENT_SECRET=djchmSAdeBonqrBcuZgQ-Q1uPFw88vJg5Qe53mA04gwjoPatOfwdgqmjbskJ34L0

### AWS Credentials

`terraform` will use credentials set in your environment or through other means as described in the [Terraform documentation](https://www.terraform.io/docs/providers/aws/index.html#environment-variables). This guide will assume you are using the "Environment Variables" method.

Add your AWS credentials as two environment variables.

Set your `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY` replacing `AAAAAA` with your own values.

In [ ]:
export AWS_ACCESS_KEY_ID=AAAAAA
export AWS_SECRET_ACCESS_KEY=AAAAA

NOTE: If you use `doormat`, you can this method.

In [ ]:
doormat --refresh && eval $(doormat aws -a se_demos_dev)

## Terraform Vault HCP Setup

This Terraform code will create:
* HCP Vault cluster
* along with the required vpcs, security groups, subnets, internet gateways, assocations, and peering

### Pre-Requisites

* Terraform ~>1.0

### Pick a Demo

At this point you get to choose your own adventure. There are a few repos that we can try out.

Go to the demo you want to try out.

* [terraform-vault-hcp-setup](#terraform-vault-hcp-setup)
  * VPC Peering
* [Andrew Demo](#Andrew-Demo)
  * Transit Gateway, Bastion Host

In [ ]:
export MAIN_DIR=$(pwd)
export GIT_DIR=hcp-vault-demo
export WORK_DIR=$MAIN_DIR/$GIT_DIR

### terraform-vault-hcp-setup

In [ ]:
# rm -rf $WORK_DIR/*
git clone https://github.com/kalenarndt/terraform-vault-hcp-setup $WORK_DIR

In [ ]:
cd $WORK_DIR

In [ ]:
cat <<EOF > terraform.tfvars
aws_product_tag     = "vault"
aws_environment_tag = "HCP"
aws_owner_tag       = "pphan"
# hcp_client_secret   = "myclientsecret"
# hcp_client_id       = "myclientid"
EOF

Now we can run our terraform project. Skip to [Provision Resources](#Provision-Resources).

### Andrew Demo

In [ ]:
rm -rf $WORK_DIR
git clone https://github.com/Andrew-Klaas/hcp-vault-demo $WORK_DIR && cd $WORK_DIR

#### Andrew Demo terraform.vars

In [ ]:
cat <<EOF > terraform.tfvars
region              = "us-west-2"
az                  = "us-west-2a"

//Instance Tags
Name                = "pphan" #"YOUR-NAME"
owner               = "pphan@hashicorp.com" # "YOUR-EMAIL"
TTL                 = 48

//Your public key will be uploaded to machine for SSH access
public_key = "$(cat ~/.ssh/id_rsa.pub)" #"ssh-rsa AA...."

EOF

Now we can run our terraform project:

### Provision Resources

terraform init, plan, apply

In [ ]:
# if you have tfswitch and need to switch to v1.0+
# tfswitch 1.0.0

In [ ]:
terraform init
# terraform plan -input=false

In [ ]:
terraform apply -input=false -auto-approve

NOTE: Deploying a Vault Cluster can take 10 minutes.

### Test Vault Functionality from Bastion Host

> NOTE: This section only works with Andrew's Demo

SSH into your newly provisioned machine using the output from the apply.

Sample Output
```shell
PUBLIC_IP = "55.199.135.119"
VAULT_ADDR = "https://demo-vault-cluster.private.vault.......63.aws.hashicorp.cloud:8200"
```

Create a bash function to reduce typing for ssh command. We are not going to do host checking, which does not work well in Jupyter.

In [ ]:
ssh_hcp() {
  ssh -i ~/.ssh/id_rsa -o StrictHostKeyChecking=no ubuntu@$(terraform output -raw PUBLIC_IP) "$@"
}

Run `vault status` on Bastion host.

In [ ]:
ssh_hcp VAULT_ADDR=$(terraform output -raw VAULT_ADDR) vault status

Sample Output
```shell
Key                      Value
---                      -----
Recovery Seal Type       shamir
Initialized              true
Sealed                   false
Total Recovery Shares    1
Threshold                1
Version                  1.7.3+ent
Storage Type             raft
Cluster Name             vault-cluster-411134ac
Cluster ID               9de2b307-b512-df16-1d7e-7347a08c95c7
HA Enabled               true
HA Cluster               https://172.25.17.216:8201
HA Mode                  active
Active Since             2021-07-15T14:46:19.053608784Z
Raft Committed Index     980
Raft Applied Index       980
Last WAL                 269
```

In [ ]:
Run `vault secrets list` on Bastion host. Replace the value of `VAULT_TOKEN` with your own.

In [ ]:
ssh_hcp VAULT_ADDR=$(terraform output -raw VAULT_ADDR) VAULT_NAMESPACE=admin VAULT_TOKEN=s.BBB.CCC vault secrets list

You can grab the Vault Admin token from the HCP portal. You can also grab it from the Terraform state file. NOTE: Make sure you secure your state file. Admin tokens are valid for 6 hours.

## (WIP) Add a Consul Cluster

Now add a Consul cluster. This is guidance from our learning portal.

In [ ]:
cd learn-hcp-consul

The challenge now is to update VPC peering with your HVN and account details following the Learning Portal instructions. https://learn.hashicorp.com/tutorials/cloud/terraform-hcp-consul-provider

In [ ]:
terraform init
terraform plan

You could also use a Terraform data source to look up the state from your other workspace. You don't need to actually have a Consul cluster created to pass this challenge check but it is a sandbox where you can experiment with Consul.

## (WIP) Exploring Service Principals

Now we'll generate three service principals. Service principals come in 3 types:

* Admin
* Contributor
* Viewer

A policy of least privilege can be used to limit API access to reading HCP state for things like Terraform data sources. HCP lets you create named service principals and then create one or more tokens to access that service principal. Be sure to delete any service principals you create during this workshop as they won't be destroyed when Instruqt expires your track. https://portal.cloud.hashicorp.com/access/service-principals

https://registry.terraform.io/providers/hashicorp/hcp/latest/docs/guides/auth

## Clean Up

### Destroy terraform provisioned resources.

In [ ]:
terraform destroy -auto-approve

## Resources

* [Terraform Vault HCP Setup](https://github.com/kalenarndt/terraform-vault-hcp-setup) by Kalen Arndt
* https://play.instruqt.com/hashicorp/tracks/hcp-vault-dynamic-credentials-alpha

## Work in Progress - Create terraform configurations

Create `main.tf` file.

#### terraform.tfvars

In [ ]:
cat <<EOF > terraform.tfvars.2
aws_product_tag     = "vault"
aws_environment_tag = "HCP"
aws_owner_tag       = "Kalen"
# hcp_client_secret   = "myclientsecret"
# hcp_client_id       = "myclientid" 

region              = "us-west-2"
az                  = "us-west-2a"

//Instance Tags
Name                = "YOUR-NAME"
owner               = "YOUR-EMAIL"
TTL                 = 48

//Your public key will be uploaded to machine for SSH access
public_key = $(cat ~/.ssh/id_rsa.pub) #"ssh-rsa AA...."
EOF

In [ ]:
cat <<EOF > main.tf.2
// Credentials can be set explicitly or via the environment variables HCP_CLIENT_ID and HCP_CLIENT_SECRET
provider "hcp" {
}

provider "aws" {
  region = var.region
  default_tags {
    tags = {
      Name  = var.Name
      owner = var.owner
      TTL   = var.TTL
    }
  }
}
EOF

cat <<EOF > versions.tf.2
terraform {
  required_version = "~>1.0.0"
  required_providers {
    aws = {
      source  = "hashicorp/aws"
      version = "~>3.51.0"
    }
    hcp = {
      source  = "hashicorp/hcp"
      version = "~>0.10.0"
    }
  }
}
EOF

In [ ]:
## variables.tf
cat <<EOF > variables.tf.2
variable "az" {
  type = string 
  default = "us-west-2a"
}

variable "region" {
  description = "The region of the HCP HVN and Vault cluster."
  type = string
  default = "us-west-2"
}

variable "Name" {
  type = string
}

variable "owner" {
  type = string
}

variable "TTL" {
  type = number
}

variable "public_key" {
  type = string
}

variable "hcp_client_id" {
  description = "Client ID used to authenticate with HCP"
  type        = string
  sensitive   = false
  default     = null
}

variable "hcp_client_secret" {
  description = "Client secret used to authenticate with HCP"
  type        = string
  sensitive   = true
  default     = null
}

variable "hcp_cluster_id" {
  description = "The ID of the HCP Vault cluster."
  type        = string
  default     = "hcp-vault-cluster"
}

variable "hcp_public_endpoint" {
  description = "Exposes the cluster to the internet. Defaults to false"
  type        = bool
  default     = false
}

variable "hcp_tier" {
  description = "Tier to provision in HCP Vault - dev, standard_small, standard_medium, standard_large"
  type        = string
  default     = "dev"
  validation {
    condition     = var.hcp_tier != "dev" || var.hcp_tier != "standard_small" || var.hcp_tier != "standard_medium" || var.hcp_tier != "standard_large"
    error_message = "The variable hcp_tier must be \"dev\", \"standard_small\", \"standard_medium\", or \"standard_large\"."
  }
}

variable "hvn_cidr" {
  type = string
  default = "172.25.16.0/20"
}

variable "hvn_id" {
  description = "The ID of the HCP HVN."
  type        = string
  default     = "hcp-vault-hvn"
}

variable "hvn_peering_id" {
  description = "The ID of the HCP peering connection."
  type        = string
  default     = "hcp-hvn-peering"
}

variable "hvn_route_id" {
  description = "The ID of the HCP HVN route."
  type        = string
  default     = "hcp-hvn-route"
}

variable "vpc_cidr" {
  type = string
  default = "10.0.1.0/24"
}








variable "cloud_provider" {
  description = "The cloud provider of the HCP HVN and Vault cluster."
  type        = string
  default     = "aws"
}

#### aws.tf

In [ ]:
## aws.tf
cat <<EOF > aws.tf.2
// create the aws vpc
resource "aws_vpc" "example" {
  cidr_block = "10.0.0.0/16"
#   tags = {
#     Name = var.aws_vpc_hvn_name
#   }
}

resource "aws_subnet" "my_subnet" {
  vpc_id            = aws_vpc.example.id
  cidr_block        = var.vpc_cidr  #default to 10.0.1.0.24
  availability_zone = var.az #defaults to us-west-2a
}

resource "aws_internet_gateway" "vpc-igw" {
  vpc_id = "${aws_vpc.example.id}"
}

resource "aws_main_route_table_association" "main-vpc" {
  vpc_id         = "${aws_vpc.example.id}"
  route_table_id = "${aws_route_table.main-rt.id}"
}

resource "aws_route_table" "main-rt" {
  vpc_id = "${aws_vpc.example.id}"

  route {
    cidr_block = var.hvn_cidr
    transit_gateway_id = "${aws_ec2_transit_gateway.example.id}"
  }

  route {
    cidr_block = "0.0.0.0/0"
    gateway_id = "${aws_internet_gateway.vpc-igw.id}"
  }

  depends_on = [ 
      aws_ec2_transit_gateway.example,
      aws_ec2_transit_gateway_vpc_attachment.example,
  ]
}

resource "aws_security_group" "main-vpc-sg" {
  name        = "${var.Name}-main-vpc-sg"
  vpc_id      = "${aws_vpc.example.id}"
  ingress {
    from_port   = 22
    to_port     = 22
    protocol    = "tcp"
    cidr_blocks = ["0.0.0.0/0"]
  }
  ingress {
    from_port   = 8 # the ICMP type number for 'Echo'
    to_port     = 0 # the ICMP code
    protocol    = "icmp"
    cidr_blocks = ["0.0.0.0/0"]
  }
  ingress {
    from_port   = 0 # the ICMP type number for 'Echo Reply'
    to_port     = 0 # the ICMP code
    protocol    = "icmp"
    cidr_blocks = ["0.0.0.0/0"]
  }
  egress {
    from_port       = 0
    to_port         = 0
    protocol        = "-1"
    cidr_blocks     = ["0.0.0.0/0"]
  }
  tags = {
  }
}

###########################
# AWS Transit Gateway
#
# Connects HVN to VPC
###########################
resource "aws_ec2_transit_gateway_vpc_attachment" "example" {
  subnet_ids         = [aws_subnet.my_subnet.id]
  transit_gateway_id = aws_ec2_transit_gateway.example.id
  vpc_id             = aws_vpc.example.id
  depends_on = [ 
    aws_ec2_transit_gateway.example, 
  ]
}

## Fetching AMI info
data "aws_ami" "ubuntu" {
  most_recent = true
  filter {
    name   = "name"
    values = ["ubuntu/images/hvm-ssd/ubuntu-bionic-18.04-amd64-server-*"]
  }
  filter {
    name   = "virtualization-type"
    values = ["hvm"]
  }
  owners = ["099720109477"] # Canonical
}

resource "aws_instance" "test-instance" {
  ami                         = "${data.aws_ami.ubuntu.id}"
  instance_type               = "t2.micro"
  subnet_id                   = "${aws_subnet.my_subnet.id}"
  vpc_security_group_ids     = [ "${aws_security_group.main-vpc-sg.id}" ]
  key_name                    = "${aws_key_pair.test-tgw-keypair.key_name}"
  associate_public_ip_address = true
  user_data                   = "${data.template_file.init.rendered}"
}

data "template_file" "init" {
  template = "${file("${path.module}/init.tpl")}"
}


resource "aws_key_pair" "test-tgw-keypair" {
  key_name   = "${var.Name}-keypair"
  public_key = "${var.public_key}"
}
EOF